In [ ]:
import os
import time
import requests
from dotenv import load_dotenv

In [ ]:
def check_file(file_path):
    # .env 파일에서 환경 변수 로드
    load_dotenv()

    # 환경 변수에서 값 읽기
    API_KEY = os.getenv("API_KEY")

    file_path = f"sample_file/{file_path}"

    print(API_KEY)

    headers = {'x-apikey': API_KEY}
    
    # 파일 업로드
    with open(file_path, 'rb') as f:
        files = {'file': (file_path, f)}
        response = requests.post('https://www.virustotal.com/api/v3/files', files=files, headers=headers)
        
        # 파일 업로드 응답 확인
        print(f"[•] 업로드 응답 상태 코드: {response.status_code}")
        print(f"[•] 응답 내용: {response.text}")  # JSON이 아닐 수도 있음

        try:
            result = response.json()
        except ValueError:
            print("❌ JSON 파싱 오류: 응답이 JSON 형식이 아닙니다.")
            return
        
        result = response.json()
        analysis_id = result['data']['id']
        print(f"[✓] 분석 ID: {analysis_id}")

    
    # 검사 결과 대기 및 확인
    result_url = f'https://www.virustotal.com/api/v3/analyses/{analysis_id}'

    while True:
        response = requests.get(result_url, headers=headers)
        data = response.json()
        status = data['data']['attributes']['status']
        print(f"[•] 검사 상태: {status}")
        
        if status == 'completed':
            break
        time.sleep(3)

    
    # 주요 결과 필드 출력
    stats = data['data']['attributes']['stats']
    print("\n[🔍 검사 결과 요약]")
    print(f"  🟥 악성(Malicious): {stats.get('malicious', 0)}")
    print(f"  🟧 의심(Suspicious): {stats.get('suspicious', 0)}")
    print(f"  🟩 무해(Harmless): {stats.get('harmless', 0)}")
    print(f"  ⏱️ 타임아웃(Timeout): {stats.get('timeout', 0)}")
    print(f"  🕵️ 탐지 안 됨(Undetected): {stats.get('undetected', 0)}")